In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# generate data
# each data is combined of different numbers of element
# each element has length of 4
# there are two class, class 0 the entries in element are totally radom
# class 1 the entries are continuous
num_sample = 2000
elementLen = 4
dataMLen = 12
dataset = []
label = []
seqLen = []
for i in range(num_sample):
    length = np.random.randint(1, dataMLen + 1)
    seqLen.append(length)
    # generate random element
    if np.random.random() < 0.5:
        temp = np.random.randint(1, 100, [length, elementLen])
        label.append([1., 0.])
    else:
        temp = np.array(range(1, length * 4 + 1))
        temp = temp.reshape([length, elementLen])
        label.append([0., 1.])
    temp = np.append(temp, np.zeros([dataMLen - length, elementLen]), axis=0)
    dataset.append(temp.tolist())

In [3]:
# separate into training dataset and testing
ratio = 0.3
test_data = dataset[-int(ratio * num_sample):]
test_label = label[-int(ratio * num_sample):]
test_len = seqLen[-int(ratio * num_sample):]
train_data = dataset[:-int(ratio * num_sample)]
train_label = label[:-int(ratio * num_sample)]
train_len = seqLen[:-int(ratio * num_sample)]

In [4]:
# set parameter
learning_rate = 0.0001
batch_size = 128
training_iter = 1000
display_step = 100
hidden_layer = 100
classes = 2
indices = 0

In [5]:
# place holder
x = tf.placeholder('float', [None, None, elementLen], name='input2')
y = tf.placeholder('float', [None, classes], name='label')
w = tf.Variable(tf.random_normal([hidden_layer, classes]), name='weights')
b = tf.Variable(tf.random_normal([classes]), name='bias')
seqlen = tf.placeholder(tf.int32, [None])

In [6]:
# get batch
def next_batch(X, Y, length, batch_size = batch_size):
    global indices
    batch_x = X[indices: indices + batch_size]
    batch_y = Y[indices: indices + batch_size]
    batch_l = length[indices: indices + batch_size]
    indices += batch_size
    if indices >= len(X):
        indices = indices % len(X)
        batch_x.extend(X[: indices])
        batch_y.extend(Y[: indices])
        batch_l.extend(length[: indices])
    return batch_x, batch_y, batch_l

In [7]:
# define RNN
# the inpute size is [batch_size, length, element_size]
def RNN(x, seqlen, w, b):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer)
    outputs, state = tf.nn.dynamic_rnn(lstm_cell, x, seqlen, dtype=tf.float32)
    batch_size = tf.shape(outputs)[0]
    dataMLen = tf.shape(x)[1]
    index = tf.range(0, batch_size) * dataMLen + (seqlen - 1)
    output = tf.gather(tf.reshape(outputs, [-1, hidden_layer]), index)
    output = tf.nn.softmax(tf.matmul(output, w) + b)
    return output

In [8]:
# set lose function
pred = RNN(x, seqlen, w, b)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# compute the accuracy
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# check x
inp = x

init = tf.initialize_all_variables()

/Users/douzhi/Software/anaconda3/envs/cs505/lib/python3.5/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
# run network
with tf.Session() as sess:
    sess.run(init)
    for i in range(training_iter):
        batch_x, batch_y, batch_l = next_batch(train_data, train_label, train_len)
#         print(len(batch_x), len(batch_y), len(batch_l))
        inp = sess.run(inp, feed_dict={x: batch_x})
        break
        output=sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, seqlen: batch_l})
        
        if i % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, seqlen: batch_l})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, seqlen: batch_l})
            print("Iter " + str(i) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            
#     print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label, seqlen: test_len}))

In [10]:
inp.shape

(128, 12, 4)